In [105]:

import pandas as pd
import csv
import statistics

In [106]:
school_data_in = "Resources/schools_complete.csv"
student_data_in = "Resources/students_complete.csv"

In [107]:
# Read School and Student Data
school_data = pd.read_csv(school_data_in)
student_data = pd.read_csv(student_data_in)

In [108]:
school_data_full = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
data_df = pd.DataFrame(school_data_full)

data_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary Info

Included Summary info -
Total number of unique schools
Total students
Total budget
Average math score
Average reading score
% passing math (the percentage of students who passed math)
% passing reading (the percentage of students who passed reading)
% overall passing (the percentage of students who passed math AND reading)

In [109]:
#school and student counts
school_count = data_df['school_name'].nunique()
student_count = data_df['Student ID'].nunique()
#budget total
budget = data_df['budget'].unique()
total_budget = sum(budget)

#average math and reading scores
avg_math_score = data_df['math_score'].mean()
avg_reading_score = data_df['reading_score'].mean()

#% passing math (the percentage of students who passed math)
#count of students passing math (>=70)
pass_math_count = school_data_full[(school_data_full["math_score"] >= 70)].count()["student_name"]

#convert count to percentage
pass_math_percentage = pass_math_count / float(student_count) * 100
#% passing reading (the percentage of students who passed reading)
pass_reading_count = school_data_full[(school_data_full["reading_score"] >= 70)].count()["student_name"]
pass_reading_percentage = pass_reading_count / float(student_count) * 100

#% overall passing (the percentage of students who passed math AND reading)
pass_math_reading_count = school_data_full[(school_data_full["math_score"] >= 70) & (school_data_full["reading_score"] >= 70)].count()["student_name"]
overall_pass_rate = pass_math_reading_count /  float(student_count) * 100

print(f"School count = {school_count}")
print(f"Student count = {student_count}")
print(f"Total budget = {total_budget}")
print(f"Avg math score = {avg_math_score}")
print(f"Avg reading score = {avg_reading_score}")
print(f"Math passing rate = {pass_math_percentage}%")
print(f"Reading passing rate = {pass_reading_percentage}%")
print(f"Overall Math/Reading passing rate = {overall_pass_rate}%")


School count = 15
Student count = 39170
Total budget = 24649428
Avg math score = 78.98537145774827
Avg reading score = 81.87784018381414
Math passing rate = 74.9808526933878%
Reading passing rate = 85.80546336482001%
Overall Math/Reading passing rate = 65.17232575950983%


In [110]:
# High-level snapshot of district's key metrics
district_summary = pd.DataFrame({"Total Schools": [school_count],
                                    "Total Students": [student_count],
                                    "Total Budget": [total_budget],
                                    "Average Math Score": [avg_math_score],
                                    "Average Reading Score": [avg_reading_score],
                                    "% Passing Math": [pass_math_percentage],
                                    "% Passing Reading": [pass_reading_percentage],
                                    "% Overall Passing": [overall_pass_rate]})

# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

# Display the DataFrame
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

In [111]:
#selecting school type 
school_types = school_data.set_index(["school_name"])["type"]

#calculating total student count
school_count_each = school_data_full.groupby(["school_name"]).count()["Student ID"]

#Total school and per student budgets
school_budget = school_data_full.groupby(["school_name"])["budget"].mean()
school_budget_student = school_budget/school_count_each

#Average Math and Reading Scores
school_math = school_data_full.groupby(["school_name"])["math_score"].mean()
school_reading = school_data_full.groupby(["school_name"])["reading_score"].mean()

#Count of schools with passing math scores and percentage
school_pass_math = school_data_full[(school_data_full["math_score"] >= 70)]
school_pass_math = school_pass_math.groupby(["school_name"]).count()["student_name"] / school_count_each * 100

#Count of schools with passing reading scores and percentage
school_pass_reading = school_data_full[(school_data_full["reading_score"] >= 70)]
school_pass_reading = school_pass_reading.groupby(["school_name"]).count()["student_name"] / school_count_each * 100

#Overall schools that passed both math and reading
pass_math_reading =school_data_full[(school_data_full["math_score"] >= 70) & (school_data_full["reading_score"] >=70)]
overall_pass_rate = pass_math_reading.groupby(["school_name"]).count()["student_name"] / school_count_each * 100



In [112]:
#Dataframe summarizing each school
schools_summary = pd.DataFrame({"School Type" : school_types,
                                   "Total Students" : school_count_each,
                                   "Total School Budget" : school_budget,
                                   "Per Student Budget" : school_budget_student,
                                   "Average Math Score" : school_math,
                                   "Average Reading Score" : school_reading,
                                   "% Passing Math" : school_pass_math,
                                   "% Passing Reading" : school_pass_reading,
                                   "% Overall Passing" : overall_pass_rate})

#setting formatting for budgets in dollars
schools_summary["Total Students"] = schools_summary["Total Students"].map("{:,}".format)
schools_summary["Total School Budget"] = schools_summary["Total School Budget"].map("${:,.2f}".format)
schools_summary["Per Student Budget"] = schools_summary["Per Student Budget"].map("${:,.2f}".format)
schools_summary["Average Math Score"] = schools_summary["Average Math Score"].map("{:,.2f}".format)
schools_summary["Average Reading Score"] = schools_summary["Average Reading Score"].map("{:,.2f}".format)
schools_summary["% Passing Math"] = schools_summary["% Passing Math"].map("{:,.2f}%".format)
schools_summary["% Passing Reading"] = schools_summary["% Passing Reading"].map("{:,.2f}%".format)
schools_summary["% Overall Passing"] = schools_summary["% Overall Passing"].map("{:,.2f}%".format)
schools_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.05,81.03,66.68%,81.93%,54.64%
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,91.33%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,53.20%
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.10,80.75,68.31%,79.30%,54.29%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,90.60%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29,80.93,66.75%,80.86%,53.53%
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,92.51%,96.25%,89.23%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,53.51%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,66.06%,81.22%,53.54%


## Highest-Performing Schools (by % Overall Passing)

In [113]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
highest_performing = schools_summary.sort_values("% Overall Passing", ascending=False)
highest_performing.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,91.33%
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.42,83.85,93.27%,97.31%,90.95%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,90.60%
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.27,83.99,93.87%,96.54%,90.58%
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59%,95.95%,90.54%


## Bottom Performing Schools (By % Overall Passing)

In [114]:
lowest_performing = schools_summary.sort_values("% Overall Passing")
lowest_performing.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.84,80.74,66.37%,80.22%,52.99%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,53.20%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,53.51%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29,80.93,66.75%,80.86%,53.53%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,66.06%,81.22%,53.54%


## Math Scores by Grade


In [115]:
average_math_scores = student_data.groupby(['school_name', 'grade'])['math_score'].mean().reset_index()
average_math_scores.columns = ['School Name', 'Grade', 'Average Math Score']
average_math_scores_df = pd.DataFrame(average_math_scores)
pivoted_math_scores = average_math_scores_df.pivot(index='School Name', columns='Grade', values='Average Math Score').reset_index()
pivoted_math_scores.columns.name = None
columns_order = ['9th', '10th', '11th', '12th']
pivoted_math_scores = pivoted_math_scores.reindex(columns=['School Name'] + columns_order)
pivoted_math_scores


,School Name,9th,10th,11th,12th
0,Bailey High School,77.083676,76.996772,77.515588,76.492218
1,Cabrera High School,83.094697,83.154506,82.765560,83.277487
2,Figueroa High School,76.403037,76.539974,76.884344,77.151369
3,Ford High School,77.361345,77.672316,76.918058,76.179963
4,Griffin High School,82.044010,84.229064,83.842105,83.356164
5,Hernandez High School,77.438495,77.337408,77.136029,77.186567
6,Holden High School,83.787402,83.429825,85.000000,82.855422
7,Huang High School,77.027251,75.908735,76.446602,77.225641
8,Johnson High School,77.187857,76.691117,77.491653,76.863248
9,Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Scores by Grade

In [116]:
average_reading_scores = student_data.groupby(['school_name', 'grade'])['reading_score'].mean().reset_index()
average_reading_scores.columns = ['School Name', 'Grade', 'Average Reading Score']
average_reading_scores_df = pd.DataFrame(average_reading_scores)
pivoted_reading_scores = average_reading_scores_df.pivot(index='School Name', columns='Grade', values='Average Reading Score').reset_index()
pivoted_reading_scores.columns.name = None
desired_columns_order = ['9th', '10th', '11th', '12th']
pivoted_reading_scores = pivoted_reading_scores.reindex(columns=['School Name'] + desired_columns_order)
pivoted_reading_scores

,School Name,9th,10th,11th,12th
0,Bailey High School,81.303155,80.907183,80.945643,80.912451
1,Cabrera High School,83.676136,84.253219,83.788382,84.287958
2,Figueroa High School,81.198598,81.408912,80.640339,81.384863
3,Ford High School,80.632653,81.262712,80.403642,80.662338
4,Griffin High School,83.369193,83.706897,84.288089,84.013699
5,Hernandez High School,80.866860,80.660147,81.396140,80.857143
6,Holden High School,83.677165,83.324561,83.815534,84.698795
7,Huang High School,81.290284,81.512386,81.417476,80.305983
8,Johnson High School,81.260714,80.773431,80.616027,81.227564
9,Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

In [117]:
# Defining the bins 
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

# Create a copy of the school summary since it has the "Per Student Budget" 
school_spending_df = schools_summary.copy()
# Use `pd.cut` to categorize spending based on the bins.
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(school_budget_student, spending_bins, labels=labels)





In [118]:
#  Calculate averages for the desired columns. Not sure why it isn't working
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Math"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Overall Passing"].mean()


C:\Users\CCF\AppData\Local\Temp\ipykernel_24336\3491395425.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()


TypeError: agg function failed [how->mean,dtype->object]

## Scores by School Size

## Scores by School Type


## SUMMARY AND ABBREVIATED ANALYSIS

I was unable to complete the analysis because I could not figure out how to get the schools to group by the various columns for their respective budgets.

Judging only based on the top and bottom performing schools it does not appear that the budget per student of each school is as much a determining factor of math, reading or overall pass rates

Instead, it appears that the size of the schools was much more influential to the success of the students, with the higher pass rates appearing to be related to having a smaller school size.
That said, the smaller schools were all charter schools, while the larger schools were district schools. The difference in success rates might also be explained by differences in school methods or curriculum, as opposed or in addition to the school size.